# David Böhler, Nico Knünz, Julian Mathis

# Bayes Exercises


**Hints**:

In many ML libraries we use the terminology ```X``` for features and ```Y``` for labels/annotations
or target values. X and Y are often numpy arrays. Such that Y is a column vector and X is a multidimensional array with:
* the first dimension represents each sample
* the remaining index dimensions represent the feature-values (these can be multi-dimensional in case we use images)

The number of samples and the number of labels (i.e.: the number of entries in the first dimension of X and Y) must be the same.
Acessing the 13-th feature vector and corresponding class in our training data:
```python
x13 = X[12, :] # index starts with 0
y13 = Y[12]
```

Using logical indexing could come in to be really helpful here:
```python
    X[Y == 0,:]  # Fetch all features (from X) according to class label 0
    X[Y == 1,:]  # Fetch all features (from X) according to class label 1
```

Also:
* ```np.unique()```
* ```np.sum()``` (have a look at the **axis** parameter)
* ``` X.reshape()```

could be helpful.


In [1]:
# This cell generates your test values (they are the same as in the decision-tree exercise).
#

import numpy as np
# Test features from previous exercise.
#
X_train = np.array([
                    [2, 1, 3],       # Class 1: Red, Green, Blue
                    [10, 30, 20],    # Class 2: Red, Green, Blue
                    [1, 3, 2],       # Class 2: Red, Green, Blue
                    [40, 20, 60]     # Class 1: Red, Green, Blue
                 ], dtype="float32")

# Labels for each feature in X_train
#
Y_train = np.array([0, 1, 1, 0])

### Exercise 1: Implementation of calculating the prior values.

* In this exercise your job is to implement the calculation prior values for each class.



**Use the following stubs for your implementation:**


```python
def compute_priors(Y):
    """ Compute the priors per class in Y.

    Parameters
    ----------
    Y: np.array
      A one dimensional numpy array containing class labels.

    Returns
    ----------
    A dictionary containing the priors per class.

    Expected Output for X_train, Y_train
    ----------
    {0: 0.5, 1: 0.5}
    """
```






In [2]:
def compute_priors(Y):
  labels = np.unique(Y)
  #print(labels)
  dict = {}
  for y in labels:
    dict[y] = np.size(Y_train[Y == y])  / np.size(Y_train)

  return dict

In [4]:
priors = compute_priors(Y_train)
print(priors)

{0: 0.5, 1: 0.5}


### Exercise 2: Implementation of calculating the pik values.

**In this exercise your job is to implement the calculation of the probability values (pik) for each feature attribute under each class.**


**Warning:**

Be careful when computing the probabilities. We do not want larger images (more pixels and therefore more entries in a non-normalized histogram) to have a larger impact.
Have a look at the features, you might have to normalize the individual feature vectors.

**Use the following stubs for your implementation:**

```python
def compute_pik(X, Y):
   """ Compute the probabilities per feature-dimension (pik).

    Parameters
    ----------
    X: np.array
      A two dimensional numpy array containing feature vectors (in rows) for each sample
      in the training data.
    Y: np.array
      A one dimensional numpy array containing class labels.

    Returns
    ----------
    A dictionary containing the piks per class where keys are the class labels and values are the pik values.

    Expected Output for X_train, Y_train
    ----------
    {0: array([0.33333333, 0.16666667, 0.5       ]),
     1: array([0.16666667, 0.5       , 0.33333333])}
    """
```






In [7]:
def compute_pik(X, Y):
  labels = np.unique(Y)
  #print(labels)
  dict = {}
  for y in labels:
    #print(y)
    array = np.zeros(X.shape[1])
    count = 0
    for x in X[Y==y]:
      #normalize and add to zero array
      #print( x / np.sum(x))
      array = np.add(array, x / np.sum(x))
      count = count + 1
    #print(array)
    #divide for mean
    dict[y] = array / count
  return dict


In [8]:
print(compute_pik(X_train, Y_train))

{0: array([0.33333334, 0.16666667, 0.5       ]), 1: array([0.16666667, 0.5       , 0.33333334])}


### Exercise 3: Implement the naive Bayes Classifier

* Implement the Bayes classifier without using the log trick.
* Implement the Bayes classifier with using the log trick.
* Compare your results.

Hint:
* ```np.power()``` to compute $p_{ik}^{x_i}$ in a single instruction could be helpful
* ```np.product()``` to compute the product of a list of values could be helpful

**Use the following stubs for your implementation:**
```python
def classify_bayes(x, piks, priors):
  """ Compute the posterior probability for a feature vector and perform classification according to naive Bayes.

  Parameters
  ----------
  x: np.array
    A one dimensional numpy array containing a single feature vector.
  piks: dict
    A dictionary of likelihoods as computed by compute_piks()
  priors: dict
    A dictionary of priors as computed by compute_priors()

  Returns
  ----------
  Class label of the most probable class and a dictionary where the key is the class and the value is the posterior.

  Expected Output for X_test[0,:]
  ----------
  (0, {0: 0.002057613168724279, 1: 6.430041152263372e-05})
  """
  pass
```

**and**

```python
def classify_bayes_log(x, piks, priors):
  """ Compute the log-posterior probability for a feature vector and perform classification according to naive Bayes.

  Parameters
  ----------
  x: np.array
    A one dimensional numpy array containing a single feature vector.
  piks: dict
    A dictionary of likelihoods as computed by compute_piks()
  priors: dict
    A dictionary of priors as computed by compute_priors()

  Returns
  ----------
  Class label of the most probable class and a dictionary where the key is the class and the value is the posterior.

  Expected Output for X_test[0,:]
  ----------
  (0, {0: -6.1862086239004945, 1: -9.65194452670022})
  """
```




In [9]:
# Use this X_test to verify your implementation.
#
X_test = np.array([
                    [5, 0, 0],    # x1
                    [2, 1, 3],    # x2
                    [2, 7, 4]     # x3
                 ])

In [15]:
import math

def classify_bayes(x, piks, priors):
  maximum = -1.
  dict = {}
  arrayL = []
  arrayB = []
  for entry in priors.keys():
    prior = priors[entry]
    pik = piks[entry]
    #every pik to the power of its count drawn
    exponented = np.power(pik,x)
    #multiply them together
    likely = np.prod(exponented)
    #
    post = prior * likely

    dict[entry] = post
    arrayB = np.append(arrayB, post)
    arrayL = np.append(arrayL, entry)
  maxIndex = np.argmax(arrayB)
  choose = int(arrayL[maxIndex])

  return (choose, dict)

def classify_bayes_log(x, piks, priors):
  maximum = -1.
  dict = {}
  arrayL = []
  arrayB = []
  for entry in priors.keys():
    prior = priors[entry]
    pik = piks[entry]
    likelyTimesDrawn = np.log(pik) * x
    sum = np.sum(likelyTimesDrawn)
    post = math.log(prior) + sum
    dict[entry] = post
    arrayB = np.append(arrayB, post)
    arrayL = np.append(arrayL, entry)
  maxIndex = np.argmax(arrayB)
  choose = int(arrayL[maxIndex])
  return (choose, dict)

In [16]:
priors = compute_priors(Y_train)
piks = compute_pik(X_train, Y_train)

print("normal")
print(classify_bayes(X_test[0], piks, priors))
print(classify_bayes(X_test[1], piks, priors))
print(classify_bayes(X_test[2], piks, priors))
print("")
print("with log")
print(classify_bayes_log(X_test[0], piks, priors))
print(classify_bayes_log(X_test[1], piks, priors))
print(classify_bayes_log(X_test[2], piks, priors))

normal
(0, {0: 0.002057613475332553, 1: 6.430042110414228e-05})
(0, {0: 0.0011574075108876966, 1: 0.00025720168441656913})
(1, {0: 1.2403632092853037e-08, 1: 1.3395921462592534e-06})

with log
(0, {0: -6.186208474888884, 1: -9.65194437768861})
(0, {0: -6.76157267939709, 1: -8.26565001656872})
(1, {0: -18.20527649651143, 1: -13.52314535879418})


## Answer: Compare your results

The normal calculation gets really small numbers, which can be hard to compare and can create errors with underflows. In comparison the log calculation has numbers in a reasnable range, whithout the chance for underflow or overflow.

The log result is also a lot easier to read and to see which class should be assigned.